# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.

Taken from:
https://colab.research.google.com/github/ai-forever/ru-gpts/blob/master/examples/Generate_text_with_RuGPTs_HF.ipynb
## Install env

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgra

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


If code below doesn't work, check your cuda version and installation here https://pytorch.org/get-started/previous-versions/

In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 15 kB/s 
     |████████████████████████████████| 12.8 MB 68.4 MB/s 
     |████████████████████████████████| 7.6 MB 6.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [ ]:
!pip install transformers==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.5 MB/s 
     |████████████████████████████████| 880 kB 52.9 MB/s 
     |████████████████████████████████| 1.1 MB 63.3 MB/s 
     |████████████████████████████████| 2.9 MB 50.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0eaab745f5a10ce6fc2f40ba7b4602ec0e3e70cd44dc048946d9e671dc9329ab
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 686 (delta 189), reused 165 (delta 165), pack-reused 442
Receiving objects: 100% (686/686), 377.90 KiB | 11.81 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [ ]:
!mkdir models/

## Download files

In [25]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hgPQj1AQE2j56pxikDF6Vn6EAYLZXxPM' -O train.txt
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kNwEH1O4Y8bJcDMNEfWbafgivMejUkj6' -O valid.txt

# Instruction: https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99

--2022-08-18 08:51:30--  https://docs.google.com/uc?export=download&id=1hgPQj1AQE2j56pxikDF6Vn6EAYLZXxPM
Resolving docs.google.com (docs.google.com)... 74.125.195.101, 74.125.195.138, 74.125.195.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ena1a1p9c02h1bp7dcmbs6f99ct3h7mq/1660812675000/03447575829673652196/*/1hgPQj1AQE2j56pxikDF6Vn6EAYLZXxPM?e=download&uuid=5e30d1a6-07ad-4a3b-b4ce-221d96f53fb0 [following]
--2022-08-18 08:51:30--  https://doc-0g-0o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ena1a1p9c02h1bp7dcmbs6f99ct3h7mq/1660812675000/03447575829673652196/*/1hgPQj1AQE2j56pxikDF6Vn6EAYLZXxPM?e=download&uuid=5e30d1a6-07ad-4a3b-b4ce-221d96f53fb0
Resolving doc-0g-0o-docs.googleusercontent.com (doc-0g-0o-docs.googleusercontent.com)... 108.177.98.132, 2607:f8b0:

In [29]:
open('train.txt', 'r').read(1000)

'Лев Николаевич Толстой Война и мир. Том Лев Николаевич Толстой ВОЙНА И МИР Том ЧАСТЬ ПЕРВАЯ - Е , . , поместья, . , , , , , ( , ) - , , мой верный раб, . Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста (право, я верю, что он Антихрист) - я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите. Ну, здравствуйте, здравствуйте. , Я вижу, что я вас пугаю, садитесь и рассказывайте. Так говорила в июле года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп , как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех: , . (или 

## Train 
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/essays \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

08/18/2022 08:37:38 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Downloading: 100% 608/608 [00:00<00:00, 674kB/s]
Downloading: 100% 1.71M/1.71M [00:00<00:00, 5.34MB/s]
Downloading: 100% 1.27M/1.27M [00:00<00:00, 2.91MB/s]
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Downloading: 100% 551M/551M [00:07<00:00, 72.1MB/s]
08/18/2022 08:37:59 - INFO - __main__ -   Training/evalu

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/essays \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

08/18/2022 08:45:01 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=5, length=500, model_name_or_path='models/essays', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.95, padding_text='', prompt='', repetition_penalty=5.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> Утро
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
Утро было ясное. Он проснулся рано, в пять часов утра и чувствовал себя совершенно разбитым: он видел только одно - свою жену; но она улыбалась ему так нежно-сладко (какою бывает улыбка женщины), что у него не оставалось ни малейшего сомнения или даже тени страха перед нею при виде ее красивого лица с таким выражением любви к ней самой... Она говорила по французски со всеми встречными мужчинами на свете! А потом они пошли обедать ужинать. В доме был беспорядок, все были заняты работой над книгой о войне графа Безухого ; а около дома была еще большая толпа народа гулявших за д

## Zip and copy on Google drive

In [ ]:
!zip -r /content/models /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2022.08.15/ (stored 0%)
  adding: content/.config/logs/2022.08.15/13.44.40.370262.log (deflated 55%)
  adding: content/.config/logs/2022.08.15/13.44.12.999341.log (deflated 86%)
  adding: content/.config/logs/2022.08.15/13.43.32.260465.log (deflated 91%)
  adding: content/.config/logs/2022.08.15/13.44.41.125604.log (deflated 54%)
  adding: content/.config/logs/2022.08.15/13.43.53.840714.log (deflated 53%)
  adding: content/.config/logs/2022.08.15/13.44

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!cp models.zip "drive/My Drive/"

### Generate mannualy

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/essays")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/essays")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
text = "Утро"
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tok.decode(out[0])

'Утро было ясное. Он проснулся рано, в пять часов утра и чувствовал себя совершенно разбитым: он видел только одно - свою жену; но она улыбалась ему так нежно-сладко (какою бывает улыбка женщины), что у него не оставалось ни малейшего сомнения или даже тени страха перед нею при виде ее красивого лица с таким выражением любви к ней самой... Она говорила по французски со всеми встречными мужчинами на свете! А потом они пошли обедать ужинать. В доме был беспорядок, все были заняты работой над книгой о войне графа Безухого ; а около дома была еще большая толпа народа гулявших за домом гостей Петербурга : офицеров французской армии под начальством князя Курагина для того чтобы отпраздновать приезд Кутузова домой накануне смотра войск Кутузовской линии во всей его силе духа русского оружия против французов вместе взятых! И вот как раз когда князь Василий вошел опять после обеда из столовой дворца Апраксина двора ко двору великого визиря Александра Павловича Мюрата навстречу князю Василью под